In [1]:
from web3 import Web3, HTTPProvider
import numpy as np
import time
import subprocess
import csv

In [2]:
contract_abi = '[ { "inputs": [ { "internalType": "uint256", "name": "bounty", "type": "uint256" }, { "internalType": "string", "name": "proposition_question", "type": "string" } ], "name": "create_proposition", "outputs": [], "stateMutability": "nonpayable", "type": "function" }, { "inputs": [ { "internalType": "bool", "name": "is_voter", "type": "bool" } ], "name": "register_user", "outputs": [], "stateMutability": "nonpayable", "type": "function" }, { "inputs": [ { "internalType": "uint256", "name": "certify_stake", "type": "uint256" } ], "name": "request_certify", "outputs": [], "stateMutability": "nonpayable", "type": "function" }, { "inputs": [ { "internalType": "uint256", "name": "vote_stake", "type": "uint256" } ], "name": "request_vote", "outputs": [ { "internalType": "uint256", "name": "", "type": "uint256" } ], "stateMutability": "nonpayable", "type": "function" }, { "inputs": [ { "internalType": "uint256", "name": "chosen_proposition_id", "type": "uint256" }, { "internalType": "bool", "name": "certify_vote", "type": "bool" } ], "name": "submit_certification_vote", "outputs": [], "stateMutability": "nonpayable", "type": "function" }, { "inputs": [ { "internalType": "uint256", "name": "prop_id", "type": "uint256" }, { "internalType": "bool", "name": "vote", "type": "bool" } ], "name": "submit_vote", "outputs": [], "stateMutability": "nonpayable", "type": "function" }, { "inputs": [], "name": "transferToBalance", "outputs": [], "stateMutability": "payable", "type": "function" }, { "inputs": [ { "internalType": "address payable", "name": "recipient", "type": "address" }, { "internalType": "uint256", "name": "withdrawAmount", "type": "uint256" } ], "name": "withdrawFromBalance", "outputs": [], "stateMutability": "payable", "type": "function" }, { "inputs": [], "name": "checkBalance", "outputs": [ { "internalType": "uint256", "name": "", "type": "uint256" } ], "stateMutability": "view", "type": "function" }, { "inputs": [ { "internalType": "address", "name": "", "type": "address" } ], "name": "isRegisteredCertifier", "outputs": [ { "internalType": "bool", "name": "", "type": "bool" } ], "stateMutability": "view", "type": "function" }, { "inputs": [ { "internalType": "address", "name": "", "type": "address" } ], "name": "isRegisteredVoter", "outputs": [ { "internalType": "bool", "name": "", "type": "bool" } ], "stateMutability": "view", "type": "function" }, { "inputs": [ { "internalType": "uint256", "name": "", "type": "uint256" } ], "name": "outcome_list", "outputs": [ { "internalType": "uint256", "name": "", "type": "uint256" } ], "stateMutability": "view", "type": "function" }, { "inputs": [], "name": "reputation", "outputs": [ { "internalType": "uint256", "name": "", "type": "uint256" } ], "stateMutability": "view", "type": "function" } ]'

bytecode_contract = '...'

# Use this bytecode for the oracle with unchanging reputation
#bytecode_contract = '...'

In [3]:
honest_voter_accuracy = 0.8
num_props = 20
num_voters = 20
num_of_experiment = 20
stake = 25

for adversarial_control in [0, 0.05, 0.15, 0.25, 0.35, 0.4, 0.45]:
    print("Adversarial Control is: " + str(adversarial_control))
    prop_list = []
    outcomes = []
    
    fields = ['Experiment #', 'Honest Voter Accuracy', 'Adversarial Control', 'Oracle Accuracy']
    
    # create csv file to hold results
    file_name = 'ece473_results_'+str(adversarial_control) + '_' + str(honest_voter_accuracy) + '.csv'
    with open(file_name, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(fields)
    
    num_of_honest_voters = num_voters*(1 - adversarial_control)

    # Repeat simulation n times 
    for experiment in range(num_of_experiment):
        print("Experiment " + str(experiment))

        process = subprocess.Popen(["ganache-cli", "-a", "41", "-p", "8545"], stdout=subprocess.DEVNULL)
        time.sleep(5)

        web3 = Web3(HTTPProvider('HTTP://127.0.0.1:8545',  request_kwargs={'timeout': 500}))

        # source: https://dev.to/subrotokumar/how-to-deploy-a-smart-contract-on-blockchain-using-python-3ki4
        contract = web3.eth.contract(bytecode=bytecode_contract, abi=contract_abi)
        tx_hash = contract.constructor().transact({"from": web3.eth.accounts[0]})
        tx_receipt= web3.eth.wait_for_transaction_receipt(tx_hash)
        address = tx_receipt.contractAddress
        contract = web3.eth.contract(address=address, abi=contract_abi)

        # Register users to the oracle system
        submitter = web3.eth.accounts[num_voters]
        for i in range(num_voters):
            voter = web3.eth.accounts[i]
            #contract.functions.transferToBalance.transact({'from': voter, 'val': 1000000000})
            contract.functions.register_user(True).transact({'from': voter})
            time.sleep(1)

        # Loop through 50 different propositions
        for i in range(num_props):
            prop = "Prop-" + str(i)
            prop_list.append(prop)
            contract.functions.create_proposition(100, prop).transact({'from': submitter})

            # each voter votes on each proposition
            for j in range(num_voters):
                voter = web3.eth.accounts[j]
                contract.functions.request_vote(stake).transact({'from': voter})

                # honest voters who vote correctly (True) with honest_voter_accuracy
                if (j < num_of_honest_voters):
                    vote = (np.random.rand()< honest_voter_accuracy)
                    contract.functions.submit_vote(i, vote).transact({'from': voter})

                #adversarial voters who vote incorrectly (False) always
                else:
                    contract.functions.submit_vote(i, False).transact({'from': voter})

            print(contract.functions.outcome_list(i).call())
            print("Proposition " + str(i+1) + " out of " + str(num_props) + " completed.")

        # calculate accuracy of oracle
        count = 0
        for i in range(num_props):
            if contract.functions.outcome_list(i).call() == 1: count+=1
        oracle_accuracy = count/num_props

        print(oracle_accuracy)
        # write results of experiment to csv file 
        with open(file_name, 'a') as csvfile: 
            # creating a csv writer object 
            csvwriter = csv.writer(csvfile) 

            # writing the data rows
            csvwriter.writerow([experiment, honest_voter_accuracy, adversarial_control, oracle_accuracy])

        process.terminate()

Adversarial Control is: 0.05
Experiment 0
1
Proposition 1 out of 20 completed.
1
Proposition 2 out of 20 completed.
1
Proposition 3 out of 20 completed.
1
Proposition 4 out of 20 completed.
1
Proposition 5 out of 20 completed.
1
Proposition 6 out of 20 completed.
1
Proposition 7 out of 20 completed.
1
Proposition 8 out of 20 completed.
1
Proposition 9 out of 20 completed.
1
Proposition 10 out of 20 completed.
1
Proposition 11 out of 20 completed.
1
Proposition 12 out of 20 completed.
1
Proposition 13 out of 20 completed.
1
Proposition 14 out of 20 completed.
1
Proposition 15 out of 20 completed.
1
Proposition 16 out of 20 completed.
1
Proposition 17 out of 20 completed.
1
Proposition 18 out of 20 completed.
1
Proposition 19 out of 20 completed.
1
Proposition 20 out of 20 completed.
1.0
Experiment 1
1
Proposition 1 out of 20 completed.
1
Proposition 2 out of 20 completed.
1
Proposition 3 out of 20 completed.
1
Proposition 4 out of 20 completed.
1
Proposition 5 out of 20 completed.
1
Pro

1
Proposition 14 out of 20 completed.
1
Proposition 15 out of 20 completed.
1
Proposition 16 out of 20 completed.
1
Proposition 17 out of 20 completed.
1
Proposition 18 out of 20 completed.
1
Proposition 19 out of 20 completed.
1
Proposition 20 out of 20 completed.
1.0
Experiment 11
1
Proposition 1 out of 20 completed.
1
Proposition 2 out of 20 completed.
1
Proposition 3 out of 20 completed.
1
Proposition 4 out of 20 completed.
1
Proposition 5 out of 20 completed.
1
Proposition 6 out of 20 completed.
1
Proposition 7 out of 20 completed.
1
Proposition 8 out of 20 completed.
1
Proposition 9 out of 20 completed.
1
Proposition 10 out of 20 completed.
1
Proposition 11 out of 20 completed.
1
Proposition 12 out of 20 completed.
1
Proposition 13 out of 20 completed.
1
Proposition 14 out of 20 completed.
1
Proposition 15 out of 20 completed.
1
Proposition 16 out of 20 completed.
1
Proposition 17 out of 20 completed.
1
Proposition 18 out of 20 completed.
1
Proposition 19 out of 20 completed.
1
P

1
Proposition 7 out of 20 completed.
1
Proposition 8 out of 20 completed.
1
Proposition 9 out of 20 completed.
1
Proposition 10 out of 20 completed.
1
Proposition 11 out of 20 completed.
1
Proposition 12 out of 20 completed.
1
Proposition 13 out of 20 completed.
1
Proposition 14 out of 20 completed.
1
Proposition 15 out of 20 completed.
1
Proposition 16 out of 20 completed.
1
Proposition 17 out of 20 completed.
1
Proposition 18 out of 20 completed.
1
Proposition 19 out of 20 completed.
1
Proposition 20 out of 20 completed.
1.0
Experiment 2
1
Proposition 1 out of 20 completed.
1
Proposition 2 out of 20 completed.
1
Proposition 3 out of 20 completed.
1
Proposition 4 out of 20 completed.
1
Proposition 5 out of 20 completed.
1
Proposition 6 out of 20 completed.
1
Proposition 7 out of 20 completed.
1
Proposition 8 out of 20 completed.
1
Proposition 9 out of 20 completed.
1
Proposition 10 out of 20 completed.
1
Proposition 11 out of 20 completed.
1
Proposition 12 out of 20 completed.
1
Propo

1.0
Experiment 12
1
Proposition 1 out of 20 completed.
1
Proposition 2 out of 20 completed.
1
Proposition 3 out of 20 completed.
1
Proposition 4 out of 20 completed.
1
Proposition 5 out of 20 completed.
1
Proposition 6 out of 20 completed.
1
Proposition 7 out of 20 completed.
1
Proposition 8 out of 20 completed.
1
Proposition 9 out of 20 completed.
1
Proposition 10 out of 20 completed.
1
Proposition 11 out of 20 completed.
1
Proposition 12 out of 20 completed.
1
Proposition 13 out of 20 completed.
1
Proposition 14 out of 20 completed.
1
Proposition 15 out of 20 completed.
1
Proposition 16 out of 20 completed.
1
Proposition 17 out of 20 completed.
1
Proposition 18 out of 20 completed.
1
Proposition 19 out of 20 completed.
1
Proposition 20 out of 20 completed.
1.0
Experiment 13
1
Proposition 1 out of 20 completed.
1
Proposition 2 out of 20 completed.
1
Proposition 3 out of 20 completed.
1
Proposition 4 out of 20 completed.
1
Proposition 5 out of 20 completed.
1
Proposition 6 out of 20 co